In [48]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [49]:
# 2. Initializing SparkSession

spark = SparkSession.builder.appName("loadcustomer").getOrCreate()

Extracting files 

In [50]:
# 3. Read customer-json File with Spark DataFrame API

df_customer = spark.read.load(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_custmer.json",\
     format="json", header = True,inferSchema = True)

# df_customer = spark.read.json(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_custmer.json")

df_customer.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [51]:
df_customer['STREET_NAME', 'APT_NO'].show()


+-----------------+------+
|      STREET_NAME|APT_NO|
+-----------------+------+
|Main Street North|   656|
|    Redwood Drive|   829|
| 12th Street East|   683|
|Country Club Road|   253|
|   Madison Street|   301|
|   Colonial Drive|     3|
|   Belmont Avenue|    84|
|     Oxford Court|   728|
|    Forest Street|    81|
|     Court Street|   561|
|    Cypress Court|   622|
|  8th Street West|   924|
|      East Avenue|   611|
|         Route 44|   680|
|    Warren Street|    71|
|     Jones Street|   195|
|       New Street|   500|
|  Division Street|   989|
|     Maple Street|   810|
|       Eagle Road|   649|
+-----------------+------+
only showing top 20 rows



In [18]:
# load credit and branch json files

# df_credit = spark.read.json(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_credit.json")
# df_branch = spark.read.json(r"C:\Users\Learner_XZHCG217\Desktop\TEK-Dataengineering\Git\Capstone\files\cdw_sapp_branch.json")

In [52]:
# show schema for 3 json files

df_customer.printSchema()
# df_credit.printSchema()
# df_branch.printSchema()

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)



In [ ]:
df_customer.show()
# df_credit.show()
# df_branch.show()

Cleaning Customer Data

In [54]:
# FIRST NAME--- Convert the first letter in Name into UpperCase
df_customer = df_customer.withColumn('FIRST_NAME', initcap(df_customer.FIRST_NAME))

# MIDDLE NAME 
df_customer = df_customer.withColumn('MIDDLE_NAME', lower(df_customer.MIDDLE_NAME))

# LAST NAME
df_customer = df_customer.withColumn('LAST_NAME', initcap(df_customer.LAST_NAME))

df_customer.show(5)

# we change few rows in file and verified

+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States|WDunham@example.co

In [55]:
# Concatenate APT_NO and STREET_NAME with "space and , " after APT_NO

df_customer = df_customer.withColumn('FULL_STREET_ADDRESS', concat(df_customer['APT_NO'], lit(', '), df_customer['STREET_NAME']))
df_customer['FULL_STREET_ADDRESS', 'APT_NO', 'STREET_NAME'].show()


+--------------------+------+-----------------+
| FULL_STREET_ADDRESS|APT_NO|      STREET_NAME|
+--------------------+------+-----------------+
|656, Main Street ...|   656|Main Street North|
|  829, Redwood Drive|   829|    Redwood Drive|
|683, 12th Street ...|   683| 12th Street East|
|253, Country Club...|   253|Country Club Road|
| 301, Madison Street|   301|   Madison Street|
|   3, Colonial Drive|     3|   Colonial Drive|
|  84, Belmont Avenue|    84|   Belmont Avenue|
|   728, Oxford Court|   728|     Oxford Court|
|   81, Forest Street|    81|    Forest Street|
|   561, Court Street|   561|     Court Street|
|  622, Cypress Court|   622|    Cypress Court|
|924, 8th Street West|   924|  8th Street West|
|    611, East Avenue|   611|      East Avenue|
|       680, Route 44|   680|         Route 44|
|   71, Warren Street|    71|    Warren Street|
|   195, Jones Street|   195|     Jones Street|
|     500, New Street|   500|       New Street|
|989, Division Street|   989|  Division 

In [56]:
# drop  and STREET_NAME from the Data frame

df_customer = df_customer.drop('APT_NO', 'STREET_NAME')

df_customer.show(5)

+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|656, Main Street ...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|  829, Redwood Drive|
|4210653310116272|     Huntley|United States|WDunham@example.com|   1243018|        IL|   

In [57]:
# change data type of phone number to string "not VARCHAR(as in schema its string)"

df_customer = df_customer.withColumn('CUST_PHONE', df_customer['CUST_PHONE'].cast('string'))
# df_cust = df_cust.select(df_cust['CUST_PHONE'].cast('string')) # with select the data frame has one column
df_customer.printSchema()
# above code "CUST_PHONE" is String


root
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = true)



In [58]:
# change phone number (XXX)XXX-XXXX

df_customer = df_customer .withColumn('CUST_PHONE', concat(lit('(555)'), df_customer ['CUST_PHONE'].substr(1,3), lit('-'), df_customer ['CUST_PHONE'].substr(4,4)))
#df_customer['CUST_PHONE'].show(5)
df_customer.show(5)

# if you run the cell twice.... the phone number changes to all 5's

+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(555)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|656, Main Street ...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(555)123-8933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|  829, Redwood Drive|
|4210653310116272|     Huntley|United States|WDunham@example.com|(555)124-3

In [59]:
df_customer.select("CUST_PHONE").show(5)

+-------------+
|   CUST_PHONE|
+-------------+
|(555)123-7818|
|(555)123-8933|
|(555)124-3018|
|(555)124-3215|
|(555)124-2074|
+-------------+
only showing top 5 rows

